In [ ]:
import polars as pl
import networkx as nx
import time
import rtsvg
from linknode_graph_patterns import LinkNodeGraphPatterns
from convey_proximity_layout import ConveyProximityLayout
rt = rtsvg.RACETrack()
def graphAsDataFrame(_g_):
    _lu_ = {'fm':[], 'to':[]}
    for _e_ in _g_.edges: _lu_['fm'].append(_e_[0]), _lu_['to'].append(_e_[1])
    return pl.DataFrame(_lu_)
_patterns_ = LinkNodeGraphPatterns()
_tiles_    = []
for i in range(len(_patterns_)):
    _type_  = _patterns_[i]
    g       = _patterns_.createPattern(_type_)
    _dists_ =  dict(nx.all_pairs_dijkstra_path_length(g))
    _lms_  = rt.identifyLandmarks(g)
    _node_colors_ = {}
    for _lm_ in _lms_[0]: _node_colors_[_lm_] = 'red'
    df     = graphAsDataFrame(g)
    params = {'df':df, 'relationships':[('fm','to')], 'w':256, 'h':256, 'node_size':'small', 'link_arrow':False, 'node_color':_node_colors_}

    t0 = time.time()
    pos_nx = nx.spring_layout(g)
    t1 = time.time()
    _tiles_.append(rt.titleSVG(rt.link(pos=pos_nx,  **params), f'{_type_} Spring (NX) {t1-t0:.2f}s'))
    
    t0 = time.time()
    pos_ref = rt.springLayout(g)
    t1 = time.time()
    _tiles_.append(rt.titleSVG(rt.link(pos=pos_ref, **params), f'Spring (RT) {t1-t0:.2f}s'))

    t0 = time.time()
    pfdl_k0    = rtsvg.PolarsForceDirectedLayout(g, k=0)
    pos_psl_k0 = pfdl_k0.results()
    t1 = time.time()
    _tiles_.append(rt.titleSVG(rt.link(pos=pos_psl_k0, **params), f'Polar_K_0 (RT) {t1-t0:.2f}s'))
                   
    t0 = time.time()
    pfdl_k1    = rtsvg.PolarsForceDirectedLayout(g, k=1)
    pos_psl_k1 = pfdl_k1.results()
    t1 = time.time()
    _tiles_.append(rt.titleSVG(rt.link(pos=pos_psl_k1, **params), f'Polar_K_1 (RT) {t1-t0:.2f}s'))

    t0 = time.time()
    cpl_k0    = ConveyProximityLayout(g, use_resistive_distances=False, k=0)
    pos_cpl_k0 = cpl_k0.results()
    t1 = time.time()
    _tiles_.append(rt.titleSVG(rt.link(pos=pos_cpl_k0, **params), f'CPL_K_0 (RT) {t1-t0:.2f}s'))

    t0 = time.time()
    cpl_k1    = ConveyProximityLayout(g, use_resistive_distances=False, k=0)
    pos_cpl_k1 = cpl_k1.results()
    t1 = time.time()
    _tiles_.append(rt.titleSVG(rt.link(pos=pos_cpl_k1, **params), f'CPL_K_0 (RT) {t1-t0:.2f}s'))

rt.table(_tiles_, per_row=6, spacer=10)